In [59]:
import pandas as pd
import networkx as nx
import os
import zipfile
import matplotlib as plt

In [2]:
pd.set_option('display.max_colwidth', -1)

/home/jlealtru/anaconda3/envs/torch/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [7]:
# unzip all the files in the folder, but first check if the zip files are there
# todo
for i in os.listdir('../data/capital_bikes/'):
    if i.endswith('zip'):
        with zipfile.ZipFile('../data/capital_bikes/'+i) as item: # treat the file as a zip
            item.extractall('../data/capital_bikes/')

SyntaxError: invalid syntax (<ipython-input-7-4717da98713b>, line 4)

In [3]:
# read all the files
list_dataframes = []

for i in os.listdir('../data/capital_bikes/'):
    if i.endswith('csv'):
        a = pd.read_csv('../data/capital_bikes/'+i)
        list_dataframes.append(a)

In [9]:
list_dataframes[0]

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,257,2015-07-01 00:00:25,2015-07-01 00:04:43,31116,California St & Florida Ave NW,31117,15th & Euclid St NW,W21516,Member
1,2683,2015-07-01 00:00:26,2015-07-01 00:45:10,31247,Jefferson Dr & 14th St SW,31247,Jefferson Dr & 14th St SW,W00476,Casual
2,2349,2015-07-01 00:02:39,2015-07-01 00:41:49,31247,Jefferson Dr & 14th St SW,31247,Jefferson Dr & 14th St SW,W20384,Casual
3,826,2015-07-01 00:02:49,2015-07-01 00:16:36,31229,New Hampshire Ave & T St NW,31602,Park Rd & Holmead Pl NW,W00972,Member
4,1192,2015-07-01 00:02:52,2015-07-01 00:22:45,31245,7th & R St NW / Shaw Library,31630,15th & East Capitol St NE,W21272,Member
...,...,...,...,...,...,...,...,...,...
1056361,1538,2015-09-30 23:54:39,2015-10-01 00:20:17,31258,Lincoln Memorial,31277,17th & G St NW,W21241,Casual
1056362,1474,2015-09-30 23:54:53,2015-10-01 00:19:27,31258,Lincoln Memorial,31277,17th & G St NW,W21239,Casual
1056363,705,2015-09-30 23:56:33,2015-10-01 00:08:19,31201,15th & P St NW,31275,New Hampshire Ave & 24th St NW,W21283,Member
1056364,1068,2015-09-30 23:56:47,2015-10-01 00:14:36,31101,14th & V St NW,31615,6th & H St NE,W21169,Member


In [4]:
# concatenate the list of dataframes
bike_trips_data = pd.concat(list_dataframes)

In [5]:
# print len
print(len(bike_trips_data))

27637242


In [6]:
bike_trips_data.head()

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type,ride_id,...,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,is_equity
0,257.0,2015-07-01 00:00:25,2015-07-01 00:04:43,31116.0,California St & Florida Ave NW,31117.0,15th & Euclid St NW,W21516,Member,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2683.0,2015-07-01 00:00:26,2015-07-01 00:45:10,31247.0,Jefferson Dr & 14th St SW,31247.0,Jefferson Dr & 14th St SW,W00476,Casual,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2349.0,2015-07-01 00:02:39,2015-07-01 00:41:49,31247.0,Jefferson Dr & 14th St SW,31247.0,Jefferson Dr & 14th St SW,W20384,Casual,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,826.0,2015-07-01 00:02:49,2015-07-01 00:16:36,31229.0,New Hampshire Ave & T St NW,31602.0,Park Rd & Holmead Pl NW,W00972,Member,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1192.0,2015-07-01 00:02:52,2015-07-01 00:22:45,31245.0,7th & R St NW / Shaw Library,31630.0,15th & East Capitol St NE,W21272,Member,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
bike_trips_data_sub = bike_trips_data[pd.DatetimeIndex(bike_trips_data['Start date']).year<=2019]
bike_trips_data_sub

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type,ride_id,...,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,is_equity
0,257.0,2015-07-01 00:00:25,2015-07-01 00:04:43,31116.0,California St & Florida Ave NW,31117.0,15th & Euclid St NW,W21516,Member,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2683.0,2015-07-01 00:00:26,2015-07-01 00:45:10,31247.0,Jefferson Dr & 14th St SW,31247.0,Jefferson Dr & 14th St SW,W00476,Casual,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2349.0,2015-07-01 00:02:39,2015-07-01 00:41:49,31247.0,Jefferson Dr & 14th St SW,31247.0,Jefferson Dr & 14th St SW,W20384,Casual,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,826.0,2015-07-01 00:02:49,2015-07-01 00:16:36,31229.0,New Hampshire Ave & T St NW,31602.0,Park Rd & Holmead Pl NW,W00972,Member,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1192.0,2015-07-01 00:02:52,2015-07-01 00:22:45,31245.0,7th & R St NW / Shaw Library,31630.0,15th & East Capitol St NE,W21272,Member,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892814,517.0,2014-06-30 23:56:05,2014-07-01 00:04:42,31201.0,15th & P St NW,31509.0,New Jersey Ave & R St NW,W20754,Member,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
892815,432.0,2014-06-30 23:56:28,2014-07-01 00:03:41,31612.0,D St & Maryland Ave NE,31619.0,Lincoln Park / 13th & East Capitol St NE,W21613,Casual,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
892816,202.0,2014-06-30 23:56:52,2014-07-01 00:00:15,31264.0,6th St & Indiana Ave NW,31232.0,7th & F St NW / National Portrait Gallery,W20474,Member,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
892817,530.0,2014-06-30 23:57:42,2014-07-01 00:06:32,31019.0,Wilson Blvd & N Edgewood St,31015.0,Rosslyn Metro / Wilson Blvd & Ft Myer Dr,W20720,Casual,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# we subset to get the data about bike trips
print(len(bike_trips_data_sub))
bike_trips_data_graph = bike_trips_data_sub[['Start station number','End station number']]
bike_trips_data_graph['trips'] = 1
bike_trips_data_graph = bike_trips_data_graph.groupby(['Start station number',
                                                       'End station number']).count().reset_index()
print(len(bike_trips_data_graph))

25702099


/home/jlealtru/anaconda3/envs/torch/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


117849


In [33]:
#bike_trips_data_sub.columns
bike_trips_data_graph.head()

,Start station number,End station number,trips
0,0.0,0.0,9
1,0.0,31014.0,8
2,0.0,31015.0,1
3,0.0,31016.0,1
4,0.0,31027.0,1


In [42]:
# filter trips within the same station
print(len(bike_trips_data_graph))
bike_trips_data_graph = bike_trips_data_graph[bike_trips_data_graph['Start station number']!=
                                              bike_trips_data_graph['End station number']]
bike_trips_data_graph = bike_trips_data_graph.reset_index()
bike_trips_data_graph = bike_trips_data_graph.drop(['index'], axis=1)
#bike_trips_data_graph. = bike_trips_data_graph.reset_index()
print(len(bike_trips_data_graph))

117849
117267


In [47]:
origin = bike_trips_data_graph['Start station number'].tolist()
destination = bike_trips_data_graph['End station number'].tolist()
weigth = bike_trips_data_graph['trips'].tolist()

In [53]:
list_edges = tuple(zip(origin, destination, weigth)) 
list_edges[0]

(0.0, 31014.0, 8)

In [54]:
graph_bikes = nx.DiGraph()
graph_bikes.add_weighted_edges_from(list_edges)

In [75]:

# verify that we're using the correct version of StellarGraph for this notebook
import stellargraph as sg

print(graph_bikes.info())

AttributeError: 'DiGraph' object has no attribute 'info'

In [81]:
bike_trips_data_graph = bike_trips_data_graph.rename(columns = {'Start station number':'source', 
                                        'End station number':'target', 
                                        'trips':'weight'})
graph_bikes = sg.StellarDiGraph(edges=bike_trips_data_graph)


In [83]:
graph_bikes.info()
from stellargraph.data import BiasedRandomWalk

rw = BiasedRandomWalk(graph_bikes)


In [84]:
walks = rw.run(
    nodes=list(graph_bikes.nodes()),  # root nodes
    length=100,  # maximum length of a random walk
    n=10,  # number of random walks per root node
    p=0.5,  # Defines (unormalised) probability, 1/p, of returning to source node
    q=2.0,  # Defines (unormalised) probability, 1/q, for moving away from source node
)
print("Number of random walks: {}".format(len(walks)))

Number of random walks: 5840


In [88]:
from gensim.models import Word2Vec

str_walks = [[str(n) for n in walk] for walk in walks]
model = Word2Vec(str_walks, size=128, window=5, min_count=0, sg=1, workers=2, iter=15)

In [89]:
model.wv["0.0"].shape

(128,)

In [93]:
# Retrieve node embeddings and corresponding subjects
node_ids = model.wv.index2word  # list of node IDs
node_embeddings = (
    model.wv.vectors
)  # numpy.ndarray of size number of nodes times embeddings dimensionality

In [96]:
model.wv.most_similar('0.0')

[('32071.0', 0.2767222225666046),
 ('31067.0', 0.24405089020729065),
 ('31107.0', 0.23405049741268158),
 ('31614.0', 0.21143442392349243),
 ('32416.0', 0.2099815309047699),
 ('31916.0', 0.2046123594045639),
 ('31656.0', 0.19007593393325806),
 ('31414.0', 0.1797153800725937),
 ('32036.0', 0.17934858798980713),
 ('31632.0', 0.17898760735988617)]

In [56]:
# get number of nodes
graph_bikes.number_of_nodes()

584

In [60]:
nx.draw_kamada_kawai(graph_bikes)

Error in callback <function flush_figures at 0x7f15e52aac20> (for post_execute):


KeyboardInterrupt: 

In [63]:
betweeness_graph_bike = nx.betweenness_centrality(graph_bikes, 
                                               normalized=True, weight='weigth')


In [71]:
betweeness_graph_bike[32609.0]



0.00024976125449741537

In [46]:
#graph_bikes = nx.from_pandas_adjacency(bike_trips_data_graph, create_using=).
graph_bikes = nx.from_pandas_edgelist(bike_trips_data_graph, 
                                       create_using=nx.DiGraph())

KeyError: 'source'

In [32]:
bike_trips_data_sub.head()

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,is_equity


In [30]:
bike_trips_data[pd.DatetimeIndex(bike_trips_data['started_at']).year==2020]
#bike_trips_data[bike_trips_data['start_lat'].isnull()==False]
#bike_trips_data[bike_trips_data['Start station number']==43.0]
#bike_trips_data[
#    bike_trips_data['Start station'].str.startswith('Anacostia')]

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,is_equity
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40166AEE48DB264F,docked_bike,2020-07-10 20:00:46,2020-07-10 20:09:42,Potomac Ave & Half St SW,570.0,4th & M St SW,21.0,38.870201,-77.010836,38.876700,-77.017800,casual,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C2D80D65DEB2C464,docked_bike,2020-07-30 22:11:33,2020-07-30 23:32:05,Maine Ave & 9th St SW,548.0,Maine Ave & 9th St SW,548.0,38.880440,-77.025236,38.880440,-77.025236,casual,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FBE8E99BAAED0455,electric_bike,2020-07-22 09:47:52,2020-07-22 09:53:38,NaN,NaN,NaN,NaN,38.920000,-77.000000,38.920000,-77.010000,casual,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E7950C376FD263E4,electric_bike,2020-07-22 10:36:55,2020-07-22 10:49:03,NaN,NaN,NaN,NaN,38.920000,-77.010000,38.920000,-77.000000,casual,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0149F273572ED43F,electric_bike,2020-07-22 15:53:55,2020-07-22 15:58:47,NaN,NaN,NaN,NaN,38.910000,-77.050000,38.900000,-77.050000,casual,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229791,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62E64B964B60B718,docked_bike,2020-10-14 16:34:15,2020-10-14 17:22:07,Anacostia Ave & Benning Rd NE / River Terrace,43.0,Anacostia Ave & Benning Rd NE / River Terrace,43.0,38.896544,-76.960120,38.896544,-76.960120,casual,NaN
229792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,313F1242C40DB408,docked_bike,2020-10-02 19:11:09,2020-10-02 19:30:35,1st & M St NE,65.0,10th & Monroe St NE,59.0,38.905697,-77.005483,38.932514,-76.992889,member,NaN
229793,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11D0589008C626B6,docked_bike,2020-10-09 11:32:47,2020-10-09 13:02:48,National Harbor Carousel,647.0,Lincoln Memorial,277.0,38.785900,-77.016700,38.888255,-77.049437,casual,NaN
229794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D52C7FA0023CFCE4,docked_bike,2020-10-24 16:11:06,2020-10-24 18:07:59,15th St & Constitution Ave NW,531.0,Henry Bacon Dr & Lincoln Memorial Circle NW,452.0,38.892244,-77.033234,38.890539,-77.049383,casual,NaN


In [ ]:
bike_trips_data

In [ ]:
# filter
# select origin destination number of trips
# baseline statistics


In [2]:
# read 
test = pd.read_csv('../data/capital_bikes/2010-capitalbikeshare-tripdata.zip')
test.head()

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,1012,2010-09-20 11:27:04,2010-09-20 11:43:56,31208,M St & New Jersey Ave SE,31108,4th & M St SW,W00742,Member
1,61,2010-09-20 11:41:22,2010-09-20 11:42:23,31209,1st & N St SE,31209,1st & N St SE,W00032,Member
2,2690,2010-09-20 12:05:37,2010-09-20 12:50:27,31600,5th & K St NW,31100,19th St & Pennsylvania Ave NW,W00993,Member
3,1406,2010-09-20 12:06:05,2010-09-20 12:29:32,31600,5th & K St NW,31602,Park Rd & Holmead Pl NW,W00344,Member
4,1413,2010-09-20 12:10:43,2010-09-20 12:34:17,31100,19th St & Pennsylvania Ave NW,31201,15th & P St NW,W00883,Member
